In [ ]:
api_key =  #replace it with openAI api key
load_trained_data = 1
run_openAI_experiments = 1-load_trained_data
# load_prompts_from_Git = 1
# run_openAI_experiments = 0  #openAI experiments
# load_RecLLMparsed_df = load_prompts_from_Git
# load_RecLLMparsed_with_itemIds = load_prompts_from_Git

In [ ]:
!pip install openai==0.28.0
# Install PyDrive
!pip install -U -q PyDrive
!pip install tqdm
import openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00


## Utility Functions

In [ ]:
import numpy as np
import pandas as pd
def print_statistics(data):
    total_interactions = data.shape[0]
    unique_users = data['userId'].nunique()
    unique_items = data['itemId'].nunique()

    print("--------------------------")
    print("The Dataset Overview:")
    print(f" - Total Interactions: {total_interactions}")
    print(f" - |U|: {unique_users}")
    print(f" - |I|: {unique_items}")
    print(f" - |R|/|U|: {total_interactions / unique_users:.2f}")
    print(f" - |R|/|I|: {total_interactions / unique_items:.2f}")
    print(f" - |R|/(|U|*|I|): {total_interactions / (unique_users * unique_items):.10f}")
    print("--------------------------")
# print_statistics(train_df)
# print_statistics(test_df)


def sample_users(df_ratings, num_users):
    # Get a list of unique users
    unique_users = df_ratings['userId'].unique()

    # Sample a specified number of users
    sampled_users = pd.Series(unique_users).sample(n=num_users, random_state=42)

    # Get all ratings for the sampled users
    df_sampled_ratings = df_ratings[df_ratings['userId'].isin(sampled_users)]

    return df_sampled_ratings


def apply_interaction_limits(data, min_interactions, max_interactions, user_col, item_col):
    # Count the number of interactions by user
    interaction_counts = data[user_col].value_counts()

    # Filter out users outside the interaction limits
    users_to_keep = interaction_counts[(interaction_counts >= min_interactions) & (interaction_counts <= max_interactions)].index
    return data[data[user_col].isin(users_to_keep)]


def analyze_tags(df_item):
    # Count unique item IDs
    unique_item_ids = df_item['itemId'].nunique()

    # Count unique tags across all items (flattening the lists first)
    unique_tags = set(tag for sublist in df_item['tag'] for tag in sublist)
    unique_tags_count = len(unique_tags)

    # Count how many items have at least one tag
    items_with_tags = df_item[df_item['tag'].apply(lambda tags: len(tags) > 0)].shape[0]

    # Count how many items have no tags
    total_items = df_item.shape[0]
    items_without_tags = total_items - items_with_tags

    # Normalize to the entire number of items to get percentages
    items_with_tags_percentage = (items_with_tags / total_items) * 100
    items_without_tags_percentage = (items_without_tags / total_items) * 100

    # Print the results
    print(f"Total unique items (itemId): {unique_item_ids}")
    print(f"Total unique tags: {unique_tags_count}")
    print(f"Items with at least one tag: {items_with_tags} ({items_with_tags_percentage:.2f}%)")
    print(f"Items with no tags: {items_without_tags} ({items_without_tags_percentage:.2f}%)")


def check_column_for_nan(df, column_name):
    """
    Checks if a column in a DataFrame, identified by its name, contains NaN values.

    Parameters:
    df (pd.DataFrame): The DataFrame to check.
    column_name (str): The name of the column to check for NaN values.

    Returns:
    int: 1 if NaN values are found, 0 if no NaN values are present.
    """
    # Ensure the column_name exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame.")

    # Return 1 if NaN values are found, otherwise return 0
    return int(df[column_name].isna().any())


# Introduce a NaN value in the 'tags' column
data = {
    'title': ['Movie A', 'Movie B', 'Movie C'],
    'genres': ['Action', 'Drama', 'Comedy'],
    'tags': ['Tag1', np.nan, 'Tag3']  # Introducing NaN in the 'tags' column
}

df = pd.DataFrame(data)
display(df)
# Check if the 'tags' column has NaN values
result = check_column_for_nan(df, 'tags')
print(f"'tags' column has NaN values: {result}")

# Check if the 'title' column has NaN values
result = check_column_for_nan(df, 'title')
print(f"'title' column has NaN values: {result}")


,title,genres,tags
0,Movie A,Action,Tag1
1,Movie B,Drama,NaN
2,Movie C,Comedy,Tag3


'tags' column has NaN values: 1
'title' column has NaN values: 0


# Load Dataset

1.   Direct download
2.   Github pull (default - train/test)



In [ ]:
import requests
import pandas as pd

def download_csv(url, csv_file_name):
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        with open(csv_file_name, "wb") as file:
            for chunk in response.iter_content(chunk_size=128):
                file.write(chunk)
        print(f"{csv_file_name} downloaded successfully.")
        return True
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
        return False

def load_datasets():
    # URLs for the ratings, items, and users datasets
    urlRatings = "https://raw.githubusercontent.com/yasdel/RecSys_Datasets/e3a18487525d3ae8ec19b39959c1a881c383e1a8/ml_latest/df_ratings_ml-latest-small.csv"
    urlItems = "https://raw.githubusercontent.com/yasdel/RecSys_Datasets/e3a18487525d3ae8ec19b39959c1a881c383e1a8/ml_latest/df_item_ml-latest-small.csv"
    urlTags = "https://raw.githubusercontent.com/yasdel/RecSys_Datasets/21484da12f01263fc0194d12c63d796628f84137/ml_latest/df_tags_ml-latest-small.csv"

    # Local filenames
    ratings_file = "df_ratings_ml-latest-small.csv"
    items_file = "df_item_ml-latest-small.csv"
    tags_file = "df_tags_ml-latest-small.csv"

    # Download the ratings, items, and users files
    if download_csv(urlRatings, ratings_file) and download_csv(urlItems, items_file) and download_csv(urlTags, tags_file):
        column_name = ['userId', 'itemId', 'rating', 'timestamp']
        df_ratings = pd.read_csv(ratings_file, sep=',', low_memory=False, header= 0 , names=column_name)
        df_items = pd.read_csv(items_file, sep=',', low_memory=False)
        df_tags = pd.read_csv(tags_file, sep=',', low_memory=False)
        df_ratings['userId'] = df_ratings['userId'].astype(int)
        df_ratings['itemId'] = df_ratings['itemId'].astype(int)
        df_items['itemId'] = df_items['itemId'].astype(int)
        df_tags['itemId'] = df_tags['itemId'].astype(int)
        df_tags['userId'] = df_tags['userId'].astype(int)

        # Group the tags by itemId and aggregate them into a list
        tags_grouped = df_tags.groupby('itemId')['tag'].apply(list).reset_index()

        # Merge the tags with the df_item to create a new column 'tags'
        df_items_merged = pd.merge(df_items, tags_grouped, on='itemId', how='left')

        # If there are items without tags, they will have NaN in the 'tags' column; replace with an empty list
        df_items_merged['tag'] = df_items_merged['tag'].apply(lambda x: x if isinstance(x, list) else [])

        print("ml-latest-small Datasets and tags loaded successfully.")
        print("Datasets loaded successfully.")
        return df_ratings, df_items,  df_tags, df_items_merged
    else:
        print("Failed to download one or more datasets.")
        return None, None, None

DS_name = "ml-latest-small"
# Usage example for ml-latest-small ratings, items, and users datasets
df_ratings, df_items, df_tags, df_items_merged = load_datasets()

print_statistics(df_ratings)
display(df_ratings.head(10))

df_ratings_ml-latest-small.csv downloaded successfully.
df_item_ml-latest-small.csv downloaded successfully.
df_tags_ml-latest-small.csv downloaded successfully.
ml-latest-small Datasets and tags loaded successfully.
Datasets loaded successfully.
--------------------------
The Dataset Overview:
 - Total Interactions: 100836
 - |U|: 610
 - |I|: 9724
 - |R|/|U|: 165.30
 - |R|/|I|: 10.37
 - |R|/(|U|*|I|): 0.0169996831
--------------------------


,userId,itemId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [ ]:
fldAddr = "https://raw.githubusercontent.com/yasdel/providerSide_CFE/"
#itemProfAddr = "293970c381630d2d7b551c00df84e7aa6db3660a/outputs/augmented_ds/enriched_dataset_ml-latest-small_filtering_min_20_max_100_gpt-3.5-turbo.csv"
itemProfAddr = "84e287dbc227b8c8e11e17693ea43437922e0c35/outputs/augmented_ds/enriched_dataset_ml-latest-small_filtering_min_20_max_100_gpt-3.5-turbo_WithAutoTagging.csv"
df_items_enriched = pd.read_csv(fldAddr+itemProfAddr, sep=',', low_memory=False)
display(df_items_enriched)

# Check if the 'title' column has NaN values
result = check_column_for_nan(df_items_enriched, 'enriched_description')
print(f"'enriched_description' column has NaN values: {result}")


# Check if the 'title' column has NaN values
result = check_column_for_nan(df_items_enriched, 'tag')
print(f"'tag' column has NaN values: {result}")


,itemId,title,genres,tag,enriched_description,filtered_tokens,filtered_text,generated_tags
0,1500,Grosse Pointe Blank (1997),Comedy|Crime|Romance,"['hit men', 'reunion']","""Grosse Pointe Blank"" is a captivating blend o...","['forefront', 'professional', 'rollercoaster',...",forefront professional rollercoaster pointe jo...,"['debi', 'grosse', 'blank', 'pointe', 'sweethe..."
1,367,"Mask, The (1994)",Action|Comedy|Crime|Fantasy,[],"""The Mask"" (1994) is a wildly entertaining act...","['genre', 'bank', 'lesson', 'embrace', 'misuse...",genre bank lesson embrace misuse slapstick tin...,"['ipkiss', 'dorian', 'misuse', 'trickster', 't..."
2,84152,Limitless (2011),Sci-Fi|Thriller,[],"""Limitless"" (2011) is a riveting Sci-Fi thrill...","['powerful', 'smart', 'morra', 'tension', 'sky...",powerful smart morra tension skyrocket care po...,"['nootropic', 'ascent', 'morra', 'loon', 'abbi..."
3,3717,Gone in 60 Seconds (2000),Action|Crime,[],"""Gone in 60 Seconds (2000)"" is a high-octane a...","['angelina', 'auto', 'dominic', 'brother', 'st...",angelina auto dominic brother steal ensemble v...,"['vinnie', 'fifty', 'vehicle', 'auto', 'zoom',..."
4,3238,Eye of the Beholder (1999),Thriller,[],"""Eye of the Beholder"" (1999) is a riveting thr...","['ewan', 'tension', 'expert', 'ashley', 'obses...",ewan tension expert ashley obsession deepest s...,"['beholder', 'eris', 'joanna', 'ashley', 'judd..."
...,...,...,...,...,...,...,...,...
3074,5404,84 Charing Cross Road (1987),Drama|Romance,['books'],"""84 Charing Cross Road"" is a heartwarming and ...","['frank', 'unlikely', 'meaningful', 'sassy', '...",frank unlikely meaningful sassy correspondence...,"['doel', 'bookseller', 'antiquarian', 'hanff',..."
3075,53894,Sicko (2007),Documentary|Drama,[],"""Sicko"" (2007) is a thought-provoking document...","['powerful', 'profound', 'change', 'portrays',...",powerful profound change portrays care greed j...,"['sicko', 'famed', 'indictment', 'fundamental'..."
3076,1342,Candyman (1992),Horror|Thriller,[],"""Candyman"" (1992) is a chilling horror-thrille...","['genre', 'visceral', 'legend', 'mirror', 'spi...",genre visceral legend mirror spirit dark night...,"['perpetuate', 'candyman', 'whisper', 'housing..."
3077,2917,Body Heat (1981),Crime|Thriller,[],"""Body Heat"" is a sultry and gripping crime thr...","['genre', 'town', 'walker', 'heat', 'drive', '...",genre town walker heat drive richard intricate...,"['racine', 'scorch', 'matty', 'crenna', 'kasda..."


'enriched_description' column has NaN values: 0
'tag' column has NaN values: 0


In [ ]:
import pandas as pd

# Find common itemIds between df_item_enriched and df_rating
common_item_ids = pd.merge(df_items_enriched[['itemId']], df_ratings[['itemId']], on='itemId')['itemId']

# Filter df_rating to create df_rating_sampled
df_rating_sampled = df_ratings[df_ratings['itemId'].isin(common_item_ids)]
display(df_rating_sampled.head(10))
print_statistics(df_rating_sampled)


,userId,itemId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
10,1,163,5.0,964983650


--------------------------
The Dataset Overview:
 - Total Interactions: 81738
 - |U|: 610
 - |I|: 3079
 - |R|/|U|: 134.00
 - |R|/|I|: 26.55
 - |R|/(|U|*|I|): 0.0435195587
--------------------------


In [ ]:
df_items_enriched

,itemId,title,genres,tag,enriched_description,filtered_tokens,filtered_text,generated_tags
0,1500,Grosse Pointe Blank (1997),Comedy|Crime|Romance,"['hit men', 'reunion']","""Grosse Pointe Blank"" is a captivating blend o...","['forefront', 'professional', 'rollercoaster',...",forefront professional rollercoaster pointe jo...,"['debi', 'grosse', 'blank', 'pointe', 'sweethe..."
1,367,"Mask, The (1994)",Action|Comedy|Crime|Fantasy,[],"""The Mask"" (1994) is a wildly entertaining act...","['genre', 'bank', 'lesson', 'embrace', 'misuse...",genre bank lesson embrace misuse slapstick tin...,"['ipkiss', 'dorian', 'misuse', 'trickster', 't..."
2,84152,Limitless (2011),Sci-Fi|Thriller,[],"""Limitless"" (2011) is a riveting Sci-Fi thrill...","['powerful', 'smart', 'morra', 'tension', 'sky...",powerful smart morra tension skyrocket care po...,"['nootropic', 'ascent', 'morra', 'loon', 'abbi..."
3,3717,Gone in 60 Seconds (2000),Action|Crime,[],"""Gone in 60 Seconds (2000)"" is a high-octane a...","['angelina', 'auto', 'dominic', 'brother', 'st...",angelina auto dominic brother steal ensemble v...,"['vinnie', 'fifty', 'vehicle', 'auto', 'zoom',..."
4,3238,Eye of the Beholder (1999),Thriller,[],"""Eye of the Beholder"" (1999) is a riveting thr...","['ewan', 'tension', 'expert', 'ashley', 'obses...",ewan tension expert ashley obsession deepest s...,"['beholder', 'eris', 'joanna', 'ashley', 'judd..."
...,...,...,...,...,...,...,...,...
3074,5404,84 Charing Cross Road (1987),Drama|Romance,['books'],"""84 Charing Cross Road"" is a heartwarming and ...","['frank', 'unlikely', 'meaningful', 'sassy', '...",frank unlikely meaningful sassy correspondence...,"['doel', 'bookseller', 'antiquarian', 'hanff',..."
3075,53894,Sicko (2007),Documentary|Drama,[],"""Sicko"" (2007) is a thought-provoking document...","['powerful', 'profound', 'change', 'portrays',...",powerful profound change portrays care greed j...,"['sicko', 'famed', 'indictment', 'fundamental'..."
3076,1342,Candyman (1992),Horror|Thriller,[],"""Candyman"" (1992) is a chilling horror-thrille...","['genre', 'visceral', 'legend', 'mirror', 'spi...",genre visceral legend mirror spirit dark night...,"['perpetuate', 'candyman', 'whisper', 'housing..."
3077,2917,Body Heat (1981),Crime|Thriller,[],"""Body Heat"" is a sultry and gripping crime thr...","['genre', 'town', 'walker', 'heat', 'drive', '...",genre town walker heat drive richard intricate...,"['racine', 'scorch', 'matty', 'crenna', 'kasda..."


In [ ]:
import openai
import pandas as pd
import time
from tqdm import tqdm
from openai.error import RateLimitError

openai.api_key = api_key  # Replace with your actual API key

model = 'text-embedding-ada-002'  # 'text-embedding-3-large' or 'text-embedding-ada-002'

def get_embedding(text, model=model, temperature=1):
    max_retries = 5
    for retry in range(max_retries):
        try:
            response = openai.Embedding.create(
                input=text,
                model=model,
                temperature=temperature
            )
            embedding = response['data'][0]['embedding']
            return embedding
        except RateLimitError:
            print(f"Rate limit reached. Retrying in {2 ** retry} seconds...")
            time.sleep(2 ** retry)
    print("Failed to retrieve embedding after multiple retries.")
    return None

# Measure the time before starting
start_time = time.time()

# Combine title, genre, and generated tags into a single text field
df_items_enriched['combined_text_genTags'] = df_items_enriched.apply(lambda row: f"Title: {row['title']}, Genre: {row['genres']}, Tags: {row['generated_tags']}", axis=1)

# Extract embeddings from the combined text fields
embeddings = []
for text in tqdm(df_items_enriched['combined_text_genTags'], desc="Processing embeddings"):
    embeddings.append(get_embedding(text, model))
    time.sleep(0.05)  # Pause between API calls

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to process embeddings: {elapsed_time} seconds")

# Add the embeddings to the dataframe
df_items_enriched[f'embedding_combined_generatedTags{model}'] = embeddings

# Display the first few rows
display(df_items_enriched.head())

Processing embeddings:  48%|████▊     | 1491/3079 [14:49<15:47,  1.68it/s]


KeyboardInterrupt: 

In [ ]:
fileName = f'enriched_dataset_ml-latest-small_filtering_min_20_max_100_gpt-3.5-turbo_Embedding_TitleGenreGenTags_{model}.csv'
df_items_enriched.to_csv(f"{fileName}", index=False)

In [ ]:
import openai
import pandas as pd
import time
from tqdm import tqdm
from openai.error import RateLimitError

openai.api_key = api_key  # Replace with your actual API key
model = 'text-embedding-ada-002'  # 'text-embedding-3-large' or 'text-embedding-ada-002'


#df_items_enriched.drop(columns=[f'embedding_combined_generatedTags{model}'], inplace=True)

# Drop the embedding column from the dataframe if it exists
if f'embedding_combined_generatedTags{model}' in df_items_enriched.columns:
  df_items_enriched.drop(columns=[f'embedding_combined_generatedTags{model}'], inplace=True)

openai.api_key = api_key  # Replace with your actual API key

model = 'text-embedding-ada-002' #'text-embedding-3-small'  # 'text-embedding-3-large' or 'text-embedding-ada-002'

def get_embedding(text, model=model, temperature=1):
    max_retries = 5
    for retry in range(max_retries):
        try:
            response = openai.Embedding.create(
                input=text,
                model=model,
                temperature=temperature
            )
            embedding = response['data'][0]['embedding']
            return embedding
        except RateLimitError:
            print(f"Rate limit reached. Retrying in {2 ** retry} seconds...")
            time.sleep(2 ** retry)
    print("Failed to retrieve embedding after multiple retries.")
    return None

# Measure the time before starting
start_time = time.time()

# Combine title, genre, and generated tags into a single text field
df_items_enriched['combined_text_Tags'] = df_items_enriched.apply(lambda row: f"Title: {row['title']}, Genre: {row['genres']}, Tags: {row['tag']}", axis=1)

# Extract embeddings from the combined text fields
embeddings = []
for text in tqdm(df_items_enriched['combined_text_Tags'], desc="Processing embeddings"):
    embeddings.append(get_embedding(text, model))
    time.sleep(0.1)  # Pause between API calls

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to process embeddings: {elapsed_time} seconds")

# Add the embeddings to the dataframe
df_items_enriched[f'embedding_combined_Tags{model}'] = embeddings

# Display the first few rows
display(df_items_enriched.head())

In [ ]:
fileName = f'enriched_dataset_ml-latest-small_filtering_min_20_max_100_gpt-3.5-turbo_Embedding_TitleGenreTags_{model}.csv'
df_items_enriched.to_csv(f"{fileName}", index=False)